In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
pd.set_option('display.max_rows', 1000)
%matplotlib inline

In [13]:
files = {
    'ProgressiveIndex': 'progressive_index',
    'ProgressiveIndexCM': 'progressive_indexcm',
}
def read_df(experiment, name, delta='', partition=''):
    df = pd.read_csv(f"results/{name}-0.2-1024-uniform-10000000.0-{experiment}-0.0001.csv")
    repetitions = df['repetition'].max() + 1
    step = int(len(df.index)/repetitions)
    df_final = df[:step].copy().reset_index()
    for rep in range(1, repetitions):
        df_final += df[step * (rep) : step * (rep + 1)].copy().reset_index()
    
    df_final = df_final/repetitions
    
    if 'index_search_time' not in df_final:
        df_final['index_search_time'] = 0.0
    df_final['query_time'] = df_final['initialization_time'] + df_final['index_search_time'] + df_final['scan_time'] + df_final['adaptation_time']
    df_final['query_time_cumsum'] = df_final['query_time'].cumsum()
    return df_final

# Total Query Cost
Total Query Cost for different deltas

In [19]:
fig = go.Figure()

def plot_time(fig, df, name):
    fig.add_trace(go.Scatter( y=df['query_time'], mode='lines',name=name))

cols = ['2','4','8']

# for col in cols:
#     init_query = []
#     df = read_df(col, files['ProgressiveIndex'], delta=0.2, partition='1024')[:50]
#     plot_time(fig, df, col.split('c')[0]+' columns PI')
# fig.update_layout(title='Total Cost ',
#                    xaxis_title='Delta',
#                    yaxis_title='Time (seconds)',yaxis_type="log")
for col in cols:
    init_query = []
    df = read_df(col, files['ProgressiveIndexCM'], delta=0.2, partition='1024')[:200]
    plot_time(fig, df, col.split('c')[0]+' columns PICM')
fig.update_layout(title='Total Cost ',
                   xaxis_title='Query #',
                   yaxis_title='Time (seconds)',yaxis_type="log")
fig.show()